In [1]:
import numpy as np
import pandas as pd

## Buses

In [2]:
% cd C:\\Users\\sebsa\\Documents\\Centrale Paris\\3A\\OEN\\Projet Synthèse\\data\\reseau

C:\Users\sebsa\Documents\Centrale Paris\3A\OEN\Projet Synthèse\data\reseau


In [3]:
def sep_longlat(string, sep=' '):
    string = string[6:-1]
    coord = string.split(sep)
    return coord

In [4]:
cols = ['bus_id','station_id','voltage','dc','symbol','under_construction','tags','geometry']
for i in range(16):
    cols.append('fict_{0}'.format(i))

buses = pd.read_csv('buses.csv', names=cols, skiprows=1)
buses.replace('nan', np.NaN, inplace=True)
buses.replace('NaN', np.NaN, inplace=True)

In [5]:
buses.head()

,bus_id,station_id,voltage,dc,symbol,under_construction,tags,geometry,fict_0,fict_1,...,fict_6,fict_7,fict_8,fict_9,fict_10,fict_11,fict_12,fict_13,fict_14,fict_15
0,1,2,380.0,f,Substation,f,"'""mw""=>""-1""","""country""=>""IQ""","""ogc_fid""=>""342""","""visible""=>""1""",...,"""mb_symbol""=>""substation""","""annotation""=>""""'",POINT(43.7146 36.129002),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,3,380.0,f,Substation,f,"'""mw""=>""-1""","""country""=>""IQ""","""ogc_fid""=>""343""","""visible""=>""1""",...,"""mb_symbol""=>""substation""","""annotation""=>""Tamem""'",POINT(43.967285 35.46067),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,4,380.0,f,Substation,f,"'""mw""=>""-1""","""country""=>""IQ""","""ogc_fid""=>""348""","""visible""=>""1""",...,"""mb_symbol""=>""substation""","""annotation""=>""An Nasirriyah""'",POINT(44.813232 31.456782),NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,5,380.0,f,Substation,f,"'""mw""=>""-1""","""country""=>""IQ""","""ogc_fid""=>""349""","""visible""=>""1""",...,"""mb_symbol""=>""substation""","""annotation""=>""Karbala""'",POINT(43.967285 32.184911),NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,6,380.0,f,Substation,f,"'""mw""=>""-1""","""country""=>""IQ""","""ogc_fid""=>""350""","""visible""=>""1""",...,"""mb_symbol""=>""substation""","""annotation""=>""Mussaib""'",POINT(44.01123 32.704111),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
buses[buses.columns[4:]] = buses[buses.columns[4:]].astype(str)

In [7]:
cols_with_POINT = []
sum_tot = 0
for name in buses.columns[3:]:
    zou = buses[name].str.count("POINT")
    zou_1 = zou[zou == 1.]
    sum_1 = zou_1.count()
    if zou_1.shape != (0,):
        cols_with_POINT.append(name)
        sum_tot += sum_1

In [8]:
cols_with_POINT

['geometry',
 'fict_7',
 'fict_8',
 'fict_9',
 'fict_10',
 'fict_12',
 'fict_13',
 'fict_14']

In [9]:
datasets_with_POINT = []
for name in buses.columns[3:]:
    datasets_with_POINT.append(buses[buses[name].str.contains('POINT')])
    
df_with_POINT = pd.concat(datasets_with_POINT)

In [10]:
cols_final = ['bus_id', 'station_id', 'voltage', 'dc', 'symbol', 'coordinates']
df_final = pd.DataFrame(index=df_with_POINT.index, columns=cols_final)
for name in df_final.columns[:-1]:
    df_final[name] = df_with_POINT[name]

for index, row in df_with_POINT.iterrows():
    bool_POINT = row.str.contains('POINT')
    bool_POINT.fillna(value=False, inplace=True)
    df_final.set_value(index, 'coordinates', row[bool_POINT].values[0])
    
df_final.coordinates = df_final.coordinates.apply(sep_longlat)
df_final[['lon','lat']] = pd.DataFrame(df_final.coordinates.tolist(), index=df_final.index)
df_final.drop('coordinates', axis=1, inplace=True)

In [11]:
df_final.head()

,bus_id,station_id,voltage,dc,symbol,lon,lat
26,6723,7564,380.0,f,joint,-0.977046660853815,37.9617462863245
27,6724,7565,380.0,f,joint,-0.76904300000001,39.436193
28,6725,7566,380.0,f,joint,-0.296630999999993,40.195659
56,6726,7567,380.0,f,joint,21.7180373235039,42.03323513796
57,6727,7568,380.0,f,joint,22.0618967750988,42.0336041748748


In [12]:
df_final.to_csv('buses_formatted.csv')

## Links

In [26]:
links = pd.read_csv('links.csv',names=np.arange(10))
links.columns = links.iloc[0]
links = links.iloc[1:]
links.head()

link_id,src_bus_id,dst_bus_id,voltage,circuits,dc,underground,under_construction,length_m,tags,geometry
4,1,2,380,1,f,f,f,77701.030928,"'""text_""=>"" ""","""symbol""=>""380-400 kV Transmission Line"
11875,3,4,380,1,f,f,f,119309.010354,NaN,'LINESTRING(44.813232 31.456782
11,4,5,380,2,f,f,f,59205.386857,"'""text_""=>"" ""","""symbol""=>""380-400 kV Transmission Line"
10,5,6,380,2,f,f,f,91727.865688,"'""text_""=>"" ""","""symbol""=>""380-400 kV Transmission Line"
8,6,8,380,1,f,f,f,30118.378222,"'""text_""=>"" ""","""symbol""=>""380-400 kV Transmission Line"


In [27]:
links.to_csv('links_formatted.csv')